In [20]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
import pinecone
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_core.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.llms import CTransformers
from uuid import uuid4

In [21]:
load_dotenv()

True

In [22]:
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_INDEX_NAME = "medical-chatbot"

In [23]:
# Step 1: Load PDF files using DirectoryLoader and PyPDFLoader
def load_pdf(directory_path):
    loader = DirectoryLoader(directory_path, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [24]:
extracted_data = load_pdf("../data/")

In [25]:
# Step 2: Split text into chunks using RecursiveCharacterTextSplitter
def split_text_into_chunks(documents):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    text_chunks = text_splitter.split_documents(documents)
    return text_chunks

In [26]:
text_chunks = split_text_into_chunks(extracted_data)
len(text_chunks)

3367

In [27]:
# Step 3: Download and generate Hugging Face model for embeddings
def generate_embeddings_model(model_name="sentence-transformers/all-MiniLM-L6-v2"):
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    return embeddings

In [28]:
embeddings = generate_embeddings_model()

In [29]:
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(PINECONE_INDEX_NAME)
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [30]:
# Embed current loaded PDF and save embeddings in vector store
# uuids = [str(uuid4()) for _ in range(len(text_chunks))]
# vector_store.add_documents(documents=text_chunks, ids=uuids)

In [31]:
# Delete embeddings as i have alredy embedded doc
# vector_store.delete(ids=[uuids[1]])

In [32]:
# Test
results = vector_store.similarity_search(
    "What is the treatment for a common cold?",
    k=2
)
results

[Document(page_content='antibiotic several times a day to prevent infection. Ice-'),
 Document(page_content='serious if it progresses to pneumonia, therefore, antibi-otics may be required. In addition, however, there are amultitude of botanical and herbal medicines that can beformulated to treat bronchitis. Some examples includeinhaling eucalyptus or other essential oils in warm steam.Herbalists recommend a tea made of mullein ( Verbascum\nthapsus ), coltsfoot ( Tussilago farfara ), and anise seed\n(Pimpinella anisum ). Homeopathic medicine and tradi-\ntional Chinese medicine may also be very useful for')]

In [33]:
query = "What is the treatment for a common cold?"
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [34]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [35]:
llm=CTransformers(model="../model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [36]:
retriever = vector_store.as_retriever(search_kwargs={'k': 2})
qa= RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever= retriever,
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [37]:
result=qa({"query": query})
print("Response : ", result["result"])

C:\Users\PRINCE\AppData\Local\Temp\ipykernel_9492\1759232318.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result=qa({"query": query})
